In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.utils import load_img, img_to_array
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
import matplotlib.pyplot as plt

c:\Users\iskan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train, _ = tfds.load('cats_vs_dogs', split=['train[:100%]'], with_info=True, as_supervised=True)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]WARNING:absl:1738 images were corrupted and were skipped


Dataset cats_vs_dogs downloaded and prepared to C:\Users\iskan\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 224
def resize_image(img, label):
  img = tf.cast(img, tf.float32)
  img = tf.image.resize(img, (SIZE, SIZE))
  img = img / 255.0
  return img, label

In [4]:
train_resized = train[0].map(resize_image)
train_batches = train_resized.shuffle(1000).batch(16)

In [5]:
base_layers = tf.keras.applications.MobileNetV2(input_shape=(SIZE, SIZE, 3), include_top=False) # заготовленная модель, где обученны перввые слои
base_layers.trainable = False #говорим не переобучать первые слои

9406464/9406464 [==============================] - 1s 0us/step


In [6]:
model = tf.keras.Sequential([  #собираем все слои вместе
                             base_layers,#базовые слои, корторые скачали
                             GlobalAveragePooling2D(),#подобие нормализации, усредняет картинку
                             Dropout(0.2),#выключение 20% нейронов наугад
                             Dense(1)
])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [7]:
model.fit(train_batches, epochs=2)

Epoch 1/2
1454/1454 [==============================] - 272s 184ms/step - loss: 0.0697 - accuracy: 0.9724
Epoch 2/2
1454/1454 [==============================] - 268s 184ms/step - loss: 0.0440 - accuracy: 0.9841


In [15]:
for i in range(0):
    img = load_img(f'{i + 1}.jpg')
    img_array = img_to_array(img)
    img_resized, _ = resize_image(img_array, _)
    img_expended = np.expand_dims(img_resized, axis=0)
    prediction = model.predict(img_expended)[0][0]
    pred_label = 'КОТ' if prediction < 0.5 else 'СОБАКА'
    plt.figure()
    plt.imshow(img)
    plt.title(f'{pred_label} {prediction}')